
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to get PostgreSQL long running queries using unSkript legos.</b>
</div>

<br>

<center><h2>Display PostgreSQL Long Running Queries</h2></center>

# Steps Overview
    1) Get postgres long running queries by passing the interval as input
    2) Collecting PostgreSQL queries and post that to the slack channel

Here we will use unSkript Long Running PostgreSQL Queries Lego. This lego takes interval: int as input. This inputs is used to findout all the Long running queries available on the PostgreSQL database.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Any, Union

from tabulate import tabulate
from pydantic import BaseModel, Field


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        print("\n")
        print(output)
        return output
    return Printer


@legoPrinter
@beartype
def postgresql_long_running_queries(handle, interval: int = 5) -> Union[List[Any], str]:
    """postgresql_long_running_queries Runs postgres query with the provided parameters.

          :type nbParamsObj: object
          :param nbParamsObj: Object containing global params for the notebook.

          :type credentialsDict: dict
          :param credentialsDict: Dictionary of credentials info.

          :type inputParamsJson: string
          :param inputParamsJson: Json string of the input params

          :rtype: All the results of the query.
      """
    # Input param validation.

    # Multi-line will create an issue when we package the Legos.
    # Hence concatinating it into a single line.

    query = "SELECT pid, user, pg_stat_activity.query_start, now() - pg_stat_activity.query_start AS query_time, query, state " \
        " FROM pg_stat_activity WHERE state = 'active' AND (now() - pg_stat_activity.query_start) > interval '%d seconds';" % interval

    cur = handle.cursor()
    cur.execute(query)
    output = []
    res = cur.fetchall()

    data = []
    for records in res:
        result = {
            "pid": records[0],
            "user": records[1],
            "query_start": records[2],
            "query_time": records[3],
            "query": records[4],
            "state": records[5]
        }
        output.append(result)
        data.append([records[0], records[4], records[5], records[3]])

    if len(res) > 0:
        headers = ["pid", "query", "state", "duration"]
        print("\n")
        output = tabulate(data, headers=headers, tablefmt="grid")

    handle.commit()
    cur.close()
    handle.close()
    return output


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "interval": "5"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(postgresql_long_running_queries, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This inputs is used to post the message to the slack channel.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Long Running Queries : {sql_queries}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

### Conclusion
In this Runbook, we demonstrated the use of unSkript's PostgreSQL legos to run PostgreSQL Query and displays collects the long running queries from a database and sends the message to a slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com